In [2]:
# Requires
'''''
1. AWS credentials: Awskeys.txt
2. s3.py
'''''


"''\n1. AWS credentials: Awskeys.txt\n2. s3.py\n"

In [4]:
# Working copy
import shutil, os
import sys
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QMessageBox, QMenuBar
from PyQt5.QtWidgets import QFileDialog,QInputDialog
import s3
import datetime
from dateutil.tz import tzutc

Title = 'ShareBox'
currentparentpath = ''
previousparentpath = ''
boxpath = ''
orgpath = ''
org = ''
keysdatafile = ''
bucketname = ''
role = ''
droppedFiles = []

UIVersion = '2.0.07172020'
print('S3 Library for ShareBox version',s3.version())

class DropArea(QtWidgets.QPushButton):
    signal = QtCore.pyqtSignal()
    def __init__(self, parent):
        super(DropArea, self).__init__(parent)
        self.setAcceptDrops(True)

    def dragEnterEvent(self, event):
        if event.mimeData().hasUrls():
            event.acceptProposedAction()
        else:
            super(DropArea, self).dragEnterEvent(event)

    def dragMoveEvent(self, event):
        super(DropArea, self).dragMoveEvent(event)

    def dropEvent(self, event):
        global droppedFiles
        
        if event.mimeData().hasUrls():
            event.setDropAction(QtCore.Qt.CopyAction)
            event.accept()
            droppedFiles = []
            for url in event.mimeData().urls():
                file = str(url.toLocalFile())
                isFile = os.path.isfile(file)
                if isFile == True:
                    droppedFiles.append(file)
            self.signal.emit()
        else:
            super(DropArea,self).dropEvent(event)

class ShareboxClient(QtWidgets.QWidget):
    def __init__(self):
        super(ShareboxClient,self).__init__()
         
        self.setWindowFlag(QtCore.Qt.WindowMaximizeButtonHint, False)
        self.setGeometry(100,100,300,400)
        
        self.setWindowTitle(Title)
        self.setWindowIcon(QtGui.QIcon('ShareBox.png')) 
        self.setFixedSize(800, 640)

        self.btnDA = DropArea(self)
        self.btnDA.setGeometry(0, 550, 600, 90)
        self.btnDA.setText("To Upload: Drop files here or click to browse to file")
        self.btnDA.clicked.connect(self.addDocument)
        self.btnDA.signal.connect(self.drop_event)

        self.listView = QtWidgets.QListWidget(self)
        self.listView.setGeometry(QtCore.QRect(0, 50, 600, 450))
        self.listView.setObjectName("listView")
        self.listView.doubleClicked.connect(self.listSelection)
        self.listView.HideSelection = False

        self.btnBack = QtWidgets.QPushButton(self)
        self.btnBack.setGeometry(610, 50, 175, 40)
        self.btnBack.setText("Back")
        self.btnBack.clicked.connect(self.goBack)

        self.btnSync = QtWidgets.QPushButton(self)
        self.btnSync.setGeometry(610, 100, 175, 40)
        self.btnSync.setText("Sync")
        self.btnSync.clicked.connect(self.sync)
        
        self.btnForceSync = QtWidgets.QPushButton(self)
        self.btnForceSync.setGeometry(610, 150, 175, 40)
        self.btnForceSync.setText("Sync (Get All)")
        self.btnForceSync.clicked.connect(self.syncforce)

        self.btnList = QtWidgets.QPushButton(self)
        self.btnList.setGeometry(610, 200, 175, 40)
        self.btnList.setText("Refresh")
        self.btnList.clicked.connect(self.load)

        self.btnDelete = QtWidgets.QPushButton(self)
        self.btnDelete.setGeometry(610, 250, 175, 40)
        self.btnDelete.setText("Delete")
        self.btnDelete.clicked.connect(self.deleteFile)

        self.btnAdd = QtWidgets.QPushButton(self)
        self.btnAdd.setGeometry(610, 300, 175, 40)
        self.btnAdd.setText("Add Folder")
        self.btnAdd.clicked.connect(self.addFolder)
        
        self.btnAbout = QtWidgets.QPushButton(self)
        self.btnAbout.setGeometry(610, 350, 175, 40)
        self.btnAbout.setText("About")
        self.btnAbout.clicked.connect(self.about)

        self.btnExit = QtWidgets.QPushButton(self)
        self.btnExit.setGeometry(610, 590, 175, 40)
        self.btnExit.setText("Exit")
        self.btnExit.clicked.connect(self.exit)
        
        self.lblDir = QtWidgets.QLabel(self)
        self.lblDir.setGeometry(10, 480, 600, 90)
        self.lblDir.setText("")

        # Set user Credentials and Login
        self.txtuser = QtWidgets.QLineEdit(self)
        self.txtuser.setGeometry(10, 5, 175, 40)
        self.txtuser.setPlaceholderText("  Enter Username") 
        
        self.txtpwd = QtWidgets.QLineEdit(self)
        self.txtpwd.setGeometry(195, 5, 175, 40)
        self.txtpwd.setEchoMode(QtWidgets.QLineEdit.Password)
        self.txtpwd.setPlaceholderText("  Enter Password") 
        
        self.btnLogin = QtWidgets.QPushButton(self)
        self.btnLogin.setGeometry(380,5, 150, 40)
        self.btnLogin.setText("Login")
        self.btnLogin.clicked.connect(self.login)
        
        # set user local Sharebox directory
        self.chkSB = QtWidgets.QCheckBox(self)
        self.chkSB.setGeometry(600,5, 200, 40)
        self.chkSB.setText("Set Local Path")
        self.chkSB.toggled.connect(lambda:self.btnstate(self.chkSB))
        
        # Disable buttons on load
        self.btnEnable(False)
        
        # Get exisiting config values
        username = s3.getuser()
        if username.strip() !='':

            self.txtuser.setText(username)
            self.txtpwd.setText(s3.getpassword(username))
        
        boxpath = s3.getboxpath()
        
      
    def drop_event(self):
        # gets executed on drop via signal
        self.add_files()

    def add_files(self): 
        if role == 'R':
            QMessageBox.critical(self, Title, "You do not have create rights.") 
            return
        upload_lst =[]
        for file in droppedFiles:
            shutil.copy(file, currentparentpath)
            entity = os.path.basename(file)
             
            fullpath = os.path.abspath(os.path.join(currentparentpath,entity))
            EntityPath = fullpath.replace(boxpath,'').replace(entity,'')
            print('fullpath',fullpath)
            print('boxpath',boxpath)
            print('entity',entity)
            print('EntityPath',EntityPath)
            if len(EntityPath) !=0:
                if EntityPath[-1] =='\\':
                    EntityPath = EntityPath[:-1]
            EntityPath = EntityPath.replace('\\','/').replace(boxpath + '/','')       
            my_dict = {'File':entity,'SubDir':EntityPath}
           
            if s3.Add2Dictionary(EntityPath,org) == True:
                upload_lst.append(my_dict)
                self.listView.addItem('-' + entity)
            else:
                print('Add2dict failed',EntityPath,org)
        if len(upload_lst) > 0:
            try:
                s3.uploadAllFiles(keysdatafile,bucketname,upload_lst,boxpath) 
                print('Upload',upload_lst)
            except:
                QMessageBox.critical(self, Title, "Upload failed!")

    def sync(self):   
        self.download(False)
            
    def syncforce(self):   
        self.download(True)
            
    def download(self,forceDownload):   
        print(keysdatafile,bucketname,org)
        try:
            s3_file_dir_lst = s3.listS3Files(keysdatafile,bucketname,org)

            if forceDownload == True:
                s3.downloadAllFiles(keysdatafile,bucketname,s3_file_dir_lst,boxpath)
            else:
                LastRun_utc = s3.getLastRun()
                s3.downloadModifiedFilesSinceLastRun(keysdatafile,bucketname,s3_file_dir_lst,boxpath,LastRun_utc)
                s3.setLastRun()
        except:
            QMessageBox.critical(self, Title, "Upload failed!") 

    def deleteFile(self):
        if role == 'R':
            QMessageBox.critical(self, Title, "You do not have delete rights.") 
            return
        item = self.listView.currentItem()
        if item == None:
            return
        delete_lst = []
        # Get list of file in s3
        s3_file_dir_lst = s3.listS3Files(keysdatafile,bucketname,org)
        s3_files = [{'Key':item['Key'],'File':item['File'],'SubDir':item['SubDir'],'Lastmodified':item['Lastmodified']} for item in s3_file_dir_lst]

        fileparent = orgpath + self.lblDir.text()
        file = os.path.abspath(os.path.join(fileparent,item.text().replace('-','')))
        searchfilekey = org + self.lblDir.text()+ '\\' + item.text().replace('-','')
        isFile = os.path.isfile(file)
        if isFile == True:
            found = next((l for l in s3_files if searchfilekey in l['File']), None)
            if found != None:
                delete_lst.append(found)
            if len(delete_lst) > 0:
                confirm = QMessageBox.question(self, Title, "Continue with deletion?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
                if confirm == QMessageBox.Yes:
                    try:
                        s3.deleteAllFiles(keysdatafile,bucketname,delete_lst)
                        os.remove(file)
                        self.listView.takeItem(self.listView.currentRow())
                    except:
                        QMessageBox.critical(self, Title, "Delete failed!") 
                        
    def addFolder(self):
        if role == 'R':
            QMessageBox.critical(self, Title, "You do not have create rights.") 
            return
        global currentparentpath
        print('currentparentpath',currentparentpath)
        folder,result = QInputDialog.getText(self, Title, 'Enter new folder name:') 
        if result == True:
            currentparentpath = os.path.join(currentparentpath,str(folder)) 
            print('currentparentpath',currentparentpath)
            s3.make_dir(currentparentpath)
            flist = os.listdir(currentparentpath)
            self.loadlist(flist,currentparentpath)
            self.lblDir.setText(currentparentpath.replace(orgpath,''))
            
    def addDocument(self):
        global droppedFiles
        fname,result = QFileDialog.getOpenFileName(self, 'Open file','c:\\',"All files (*.*)")   
        droppedFiles = []
        droppedFiles.append(fname)
        self.add_files()

    def btnstate(self,b):
        global boxpath
        if b.text() == "Set Local Path":
            if b.isChecked() == True:
                boxpath = str(QFileDialog.getExistingDirectory(self, "Select Directory"))
                print('Set boxpath to',boxpath)
                s3.setboxpath(boxpath)
                s3.resetLastRun()
                b.setChecked(False)
                
    def goBack(self):
        global currentparentpath
        if self.lblDir.text().strip() != '':
            destination = self.lblDir.text().split('\\')
            if len(destination) >= 2:
                currentparentpath = currentparentpath.replace('\\' + str(destination[len(destination) - 1]),'')
                flist = os.listdir(currentparentpath)
                self.loadlist(flist,currentparentpath)
                self.lblDir.setText(currentparentpath.replace(orgpath,''))
                                                   
    def listSelection(self):
        self.selectionAction()
            
    def selectionAction(self):
        global currentparentpath,previousparentpath
        item = self.listView.currentItem()
        fullpath = os.path.join(currentparentpath,item.text().replace('-','').replace('+',''))
        isFile = os.path.isfile(fullpath)

        if isFile == False:
            previousparentpath = currentparentpath
            currentparentpath = os.path.join(currentparentpath,item.text().replace('+',''))
            flist = os.listdir(currentparentpath)
            self.loadlist(flist,currentparentpath)
            self.lblDir.setText(currentparentpath.replace(orgpath,''))
        else:
            self.openFile(fullpath)
        
    def about(self):
        QMessageBox.about(self, Title, "ShareBox:\n~A Dropbox inspired application\n\nVersion " + UIVersion + "\n\nFinal Project for Cloud Computing\n\nDeveloped by 'Open Group'\nTeam - Brendan T, Sumanth B, Abdullateef A, Freddie Z, and Ryan B") 
        
    def loadlist(self,flist,fullpath):
        self.listView.clear()
        for item in flist:
            fol = '+'
            if len(item.split('.')) > 1:
                fol = '-'
            self.listView.addItem(fol + item) 
            if item == org:
                self.listView.setCurrentRow(0)
                self.selectionAction()
                break
            self.listView.sortItems()
          
    def load(self):
        global currentparentpath,keysdatafile,bucketname,boxpath,orgpath,org,role
        try:
            org = s3.getorg()
            role = s3.getrole()
            boxpath = s3.getboxpath()
            self.setWindowTitle(Title + ' | ' + org + ' | ' + self.txtuser.text().strip())
            keysdatafile,bucketname,boxpath = s3.getEssentials(boxpath)
            orgpath = os.path.join(boxpath,org) 
            s3.make_dir(orgpath)
            s3.setboxpath(boxpath)
            self.btnEnable(True) 
            self.download(False)
            currentparentpath = orgpath
            flist = os.listdir(orgpath)
            self.loadlist(flist,orgpath) 
        except:
            QMessageBox.critical(self, Title, "Load failed!") 

    def exit(self):
        self.close()  
        
    def login(self):
        username = self.txtuser.text().strip()
        userpassword = self.txtpwd.text().strip()
        if username == '' or userpassword == '':
            QMessageBox.critical(self, Title, "Username and password are required!")  
        else:
            if s3.login('',username,userpassword == '') == True:
                self.load()
            else:
                QMessageBox.critical(self, Title, "Login failed!") 
                
    def btnEnable(self,value):
        self.btnSync.setEnabled(value)
        self.btnList.setEnabled(value)
        self.btnForceSync.setEnabled(value)
        self.btnBack.setEnabled(value)
        self.chkSB.setEnabled(not value)
        if role == 'R':
            self.btnAdd.setEnabled(False)
            self.btnDelete.setEnabled(False)
            self.btnDA.setEnabled(False)
        else:
            self.btnAdd.setEnabled(value)
            self.btnDA.setEnabled(value)
            self.btnDelete.setEnabled(value)

    def openFile(self,fileName):
        os.system("start " + fileName)
        

        
def main():
    app = QtWidgets.QApplication(sys.argv)
    
    form = ShareboxClient()
    form.show()
    app.exec_()
    
if __name__ == '__main__':
    main()

S3 Library for ShareBox version 2.9
awskeys.txt uab-dropbox test-organization
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Archive\Proposal.docx
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Archive\dog.jpg
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Archive\indoor.jpg
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\building.jpg
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\DesignOptions.docx
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Lab\DesignOptions.docx
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Lab\Specifications.docx
Successful File Download:  C:\Users\sumanth.bail\Documents\Test\test-organization\Pictures\Old\Candidates.docx
fullpath C:\Users\sumanth.bail\Documen